In [1]:
import torch

from transformers import BertTokenizer, BertModel

In [2]:
!pip install sentencepiece

In [3]:
import pandas as pd
df = pd.read_csv('../Data/dontpatronizeme_pcl.tsv', sep = '\t', names=['id','info','country', 'text','label'] )

In [4]:
df.head()

,id,info,country,text,label
0,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0
1,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0
2,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0
3,@@7811231,disabled,nz,Council customers only signs would be displaye...,0
4,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0


In [4]:
df['label'].value_counts()

0    8529
1     947
3     458
4     391
2     144
Name: label, dtype: int64

In [5]:
df = df.dropna(inplace = False)

df = df.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10468 entries, 0 to 10467
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       10468 non-null  object
 1   info     10468 non-null  object
 2   country  10468 non-null  object
 3   text     10468 non-null  object
 4   label    10468 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 409.0+ KB


In [6]:
# Importing the libraries needed
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
df.head(2)

In [7]:
df_final = df[['text','label']]
#df_data = data.dropna(subset=['id','info','country'])
df_final                  

df_final.shape
#df_final.columns

(10468, 2)

In [10]:
df_final['label_']= [ 0 if (y == 1 or y == 0) else 1 for y in df_final['label']]
df_final
df_final.drop('label', axis = 1, inplace= True)
df_final.head()

C:\Users\PREETA~1\AppData\Local\Temp/ipykernel_3772/4051987690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['label_']= [ 0 if (y == 1 or y == 0) else 1 for y in df_final['label']]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text,label_
0,"We 're living in times of absolute insanity , ...",0
1,"In Libya today , there are countless number of...",0
2,White House press secretary Sean Spicer said t...,0
3,Council customers only signs would be displaye...,0
4,""" Just like we received migrants fleeing El Sa...",0


In [11]:
nclasses = len(list(df_final.label_.unique()))
nclasses


2

In [12]:
my_classes = {c:i for i, c in enumerate(list(df_final.label_.unique()))}
df_final['label'] = [my_classes[l] for l in df_final.label_]

C:\Users\PREETA~1\AppData\Local\Temp/ipykernel_3772/40243744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['label'] = [my_classes[l] for l in df_final.label_]


In [13]:
df_final.label_.value_counts()

0    9475
1     993
Name: label_, dtype: int64

0    9475
1     993
Name: label_, dtype: int64

In [14]:
MAX_LEN= 512
TRAIN_BS = 8
VALID_BS = 8
EPOCHS = 1
LR =  1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [15]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [16]:
class CompanyData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):# function compulsory
        sent = str(self.df.text[index])
        sent= " ".join(review.split())
        inputs = self.tokenizer.encode_plus(sent
        , 
        None,
        add_special_tokens = True,
        max_length=self.max_len,
        padding = 'max_length',
        return_token_type_ids = True,
        truncation=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return { 'ids': torch.tensor(ids, dtype=torch.long),
               'mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.df.label_[index], dtype=torch.long)
                
               }
    
    def __len__(self):
        return self.len
        

In [17]:
train_size = 0.8
train_dataset = df_final.sample(frac= train_size, random_state=20)
test_dataset = df_final.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df_final.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CompanyData(train_dataset, tokenizer, MAX_LEN)
testing_set = CompanyData(test_dataset, tokenizer, MAX_LEN)


FULL Dataset: (10468, 3)
TRAIN Dataset: (8374, 3)
TEST Dataset: (2094, 3)


In [18]:
train_params = {'batch_size': TRAIN_BS,
               'shuffle':True,
               'num_workers': 2}
test_params = {'batch_size': VALID_BS,
              'shuffle': False,
              'num_workers': 2}

trainloader = DataLoader(training_set, **train_params)
testloader = DataLoader(testing_set, **test_params)


In [19]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self, nclasses):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, nclasses)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [20]:
model = DistillBERTClass(nclasses)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [21]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =model.parameters(), lr=LR)

In [22]:
def calculate_acc(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [23]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [24]:
writer = SummaryWriter('runs/textclassify_experiment_1')

In [25]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    
    model.train()
    for _, data in tqdm(enumerate(trainloader, 0)):
        ids = df['ids'].to(device, dtype=torch.long)
        mask = df['mask'].to(device, dtype=torch.long)
        targets = df['targets'].to(device, dtype=torch.long)
        
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_acc(big_idx, targets)
        
        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if _ % 50 == 0:
            loss_step = tr_loss/nb_tr_steps
            acc_step = (n_correct*100)/nb_tr_examples
            print(f'Training Loss per 50 steps: {loss_step}, Training Accuracy: {acc_step}')
            writer.add_scalar('training_loss', loss_step, epoch*len(trainloader) +_)
            
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
    print(f'Total Accuracy Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_acc = (n_correct*100)/nb_tr_examples
    print(f'Training Loss Epoch: {epoch_loss}, Training Accuracy Epoch: {epoch_acc}')
    
    return

In [26]:
# Validation

def valid(model, testloader):
    model.eval()
    n_correct =0; n_wrong =0; total=0
    with torch.no_grad():
        for _, data in enumerate(testloader, 0):
            ids = df['ids'].to(device, dtype=torch.long)
            mask = df['mask'].to(device, dtype=torch.long)
            targets = df['targets'].to(device, dtype=torch.long)
            
            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_acc(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)
            
            if _ % 100 == 0:
                loss_step = tr_loss/nb_tr_steps
                acc_step = (n_correct*100)/nb_tr_examples
                print(f'Validation Loss per 100 steps: {loss_step}, Validation Accuracy per: {acc_step}')   
        
    epoch_loss = tr_loss/nb_tr_steps
    epoch_acc = (n_correct*100)/nb_tr_examples
    print(f'Validation Loss Epoch: {epoch_loss}, Validation Accuracy Epoch: {epoch_acc}')
    
    return epoch_acc

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
acc = valid(model, testloader)